In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

import os
os.chdir('..')


from src.swbm import *
from src.plots import *
from src.utils import *

In [4]:
%pwd


'c:\\Users\\motze\\OneDrive\\Master\\ESM\\VegSWBM'

In [5]:
# Load data and format
input_swbm_ger = pd.read_csv('data/Data_swbm_Germany.csv')
input_ger = prepro(input_swbm_ger)

input_swbm_swe = pd.read_csv('data/Data_swbm_Sweden.csv')
input_swe = prepro(input_swbm_swe)

input_swbm_esp = pd.read_csv('data/Data_swbm_Spain.csv')
input_esp = prepro(input_swbm_esp)

sites = [input_ger, input_swe, input_esp]

In [6]:
# Load best parameters
with open('results/ger_output.json', 'r') as file:
    ger_calib = json.load(file)

with open('results/swe_output.json', 'r') as file:
    swe_calib = json.load(file)

with open('results/esp_output.json', 'r') as file:
    esp_calib = json.load(file)

calibs = [ger_calib, swe_calib, esp_calib]

In [8]:
# Create test masks
start_test = '2015-01-01'
end_test = '2020-12-31'
test_mask = (input_ger['time'] >= start_test) & (input_ger['time'] <= end_test)

start_eval = '2016-01-01'
end_eval = end_test

In [9]:
# test SWBM for all sites
swbm_param = 'b0'
evals_test = []
model_results = []
for site, calib in zip(sites, calibs):
    #break
    # get test data
    input_swbm = site[test_mask]
    input_swbm.reset_index(drop=True, inplace=True)

    # get eval mask and eval input
    eval_mask = (input_swbm['time'] >= start_eval) & (input_swbm['time'] <= end_eval)
    input_eval = input_swbm[eval_mask]
    input_eval.reset_index(drop=True, inplace=True)

    const_swbm_params = calib[0].copy()
    params_seasonal = calib[0].copy()

    # Get sinus curve for current single parameter
    params_seasonal[swbm_param] = seasonal_sinus(
        len(input_swbm),
        amplitude=calib[3]['amplitude'],
        freq=calib[3]['freq'],
        phase=calib[3]['phase'],
        center=calib[3]['center'],
        which=swbm_param)

    # Run SWBM with optimized seasonal b0
    preds_seasonal = predict_ts(input_swbm, params_seasonal)
    moists_seasonal, runoffs_seasonal, ets_seasonal, na_count = preds_seasonal

    # Save data in dataframe
    model_result = input_swbm.copy()
    model_result = pd.concat([model_result, pd.DataFrame({'moists_seasonal': moists_seasonal,
                                  'runoffs_seasonal': runoffs_seasonal,
                                  'ets_seasonal': ets_seasonal}, index=model_result.index)], axis=1)

    # cut data for evaluation
    moists_eval = moists_seasonal[eval_mask]
    runoffs_eval = runoffs_seasonal[eval_mask]
    ets_eval = ets_seasonal[eval_mask]
    
    # calculate correlation
    eval_df = eval_swbm(input_eval,
                        {'sm': moists_eval,
                        'ro': runoffs_eval,
                        'le': ets_eval},
                        swbm_param)
    
    result_test = {
                'Combination': calib[0],
                'sum_corr': eval_df['corr'].sum(),
                'SinusParameters': calib[3],
                'eval_df': eval_df,
            }

    evals_test.append(result_test)
    model_results.append(model_result)

In [10]:
evals_test

[{'Combination': {'c_s': 210, 'b0': 0.4, 'g': 0.2, 'a': 4},
  'sum_corr': 2.514752047418406,
  'SinusParameters': {'amplitude': 0.1492778497316043,
   'freq': 1.971598094574026,
   'phase': 4.50728358299142,
   'center': 0.39101391650570866},
  'eval_df':   parameter kind      corr           pval
  0        b0   sm  0.900114   0.000000e+00
  1        b0   ro  0.689366  2.336686e-155
  2        b0   le  0.925272   0.000000e+00},
 {'Combination': {'c_s': 420, 'b0': 0.4, 'g': 0.2, 'a': 8},
  'sum_corr': 1.3360927120803074,
  'SinusParameters': {'amplitude': 1.5079919848812897,
   'freq': 1.9945868187040925,
   'phase': 4.537357776131512,
   'center': -0.5272264925923769},
  'eval_df':   parameter kind      corr           pval
  0        b0   sm  0.623121  6.788451e-119
  1        b0   ro  0.203081   1.150016e-11
  2        b0   le  0.509891   1.403256e-73},
 {'Combination': {'c_s': 210, 'b0': 0.4, 'g': 0.8, 'a': 8},
  'sum_corr': 2.575536373333679,
  'SinusParameters': {'amplitude': 0.215

In [11]:
# Transform function for evaluation
def transform_evals(input_data):
    output_data = [
        input_data['Combination'],
        {'sum_corr': input_data['sum_corr']},
        {'sm_corr': input_data['eval_df']['corr'][0],
         'ro_corr': input_data['eval_df']['corr'][1],
         'le_corr': input_data['eval_df']['corr'][2]},
        input_data['SinusParameters']
    ]
    return output_data

In [15]:
# Transform evaluation
ger_test_evals = transform_evals(evals_test[0])
swe_test_evals = transform_evals(evals_test[1])
esp_test_evals = transform_evals(evals_test[2])

In [16]:
swe_test_evals

[{'c_s': 420, 'b0': 0.4, 'g': 0.2, 'a': 8},
 {'sum_corr': 1.3360927120803074},
 {'sm_corr': 0.6231205215169023,
  'ro_corr': 0.20308139419603025,
  'le_corr': 0.509890796367375},
 {'amplitude': 1.5079919848812897,
  'freq': 1.9945868187040925,
  'phase': 4.537357776131512,
  'center': -0.5272264925923769}]

In [17]:
# Save evaluations
with open('results/ger_test_eval.json', 'w') as file:
    json.dump(ger_test_evals, file)

with open('results/swe_test_eval.json', 'w') as file:
    json.dump(swe_test_evals, file)

with open('results/esp_test_eval.json', 'w') as file:
    json.dump(esp_test_evals, file)

In [19]:
# Save model predictions
model_results[0].to_csv('data/output/ger_model_test.csv', index=False)
model_results[1].to_csv('data/output/swe_model_test.csv', index=False)
model_results[2].to_csv('data/output/esp_model_test.csv', index=False)